In [3]:
import tweepy
import pandas as pd
import re
import emoji
import nltk
from nltk.corpus import stopwords
import praw
import pandas as pd
import time

In [2]:
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("english"))
reddit = praw.Reddit(
    client_id="5JisFOzxeGOQl8Uzp8u7oQ",
    client_secret="oUbmu86OO-NtBM5m24U7QsYlR2cmaQ",
    user_agent="MyRedditBot/1.0 (by u/YourRedditUsername)"
)
SUBREDDITS = ["depression", "mentalhealth", "addiction", "suicidewatch"]
KEYWORDS = ["depressed", "hopeless", "numb", "empty", "worthless", "suicidal", 
            "want to die", "can't go on", "self-harm", "overwhelmed", 
            "anxious", "panic attack", "lost", "addiction", "withdrawal", 
            "relapse", "need help"]


posts = []
post_count = 0
batch_size = 300 

for subreddit in SUBREDDITS:
    for post in reddit.subreddit(subreddit).new(limit=1000):  
        
        if any(keyword in post.title.lower() or keyword in post.selftext.lower() for keyword in KEYWORDS):
            posts.append([
                post.id, post.created_utc, post.title, post.selftext, 
                post.score, post.num_comments, post.permalink
            ])
            post_count += 1

       
        if post_count >= 20000:
            break
    
    if post_count >= 20000:
        break 

    time.sleep(5)


reddit_df = pd.DataFrame(posts, columns=["ID", "Timestamp", "Title", "Content", "Upvotes", "Comments", "URL"])


def clean_text(text):
    text = text.lower()
    text = emoji.demojize(text)  
    text = re.sub(r"http\S+|www\S+", "<URL>", text)  
    text = re.sub(r"[^a-zA-Z\s]", "", text)  
    text = " ".join(word for word in text.split() if word not in STOPWORDS) 
    return text

reddit_df["Cleaned_Title"] = reddit_df["Title"].apply(clean_text)
reddit_df["Cleaned_Content"] = reddit_df["Content"].apply(clean_text)

file_path = "reddit_crisis_posts_final.csv"
reddit_df.to_csv(file_path, index=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dines\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv("reddit_crisis_posts_final.csv")

In [5]:
df.columns

Index(['ID', 'Timestamp', 'Title', 'Content', 'Upvotes', 'Comments', 'URL',
       'Cleaned_Title', 'Cleaned_Content'],
      dtype='object')

In [6]:
df=df.drop(columns=['Content','Title','URL'])

In [7]:
df.head()

,ID,Timestamp,Upvotes,Comments,Cleaned_Title,Cleaned_Content
0,1j6gtbu,1.741439e+09,1,0,depressed lazy,got sick overnight hate life motivation anythi...
1,1j6gd5v,1.741438e+09,1,0,broke front tooth years braces making depressed,treated braces childhood teeth returned old st...
2,1j6gbtr,1.741438e+09,2,0,trapped,feel trapped im allowed sad tell stop feel alo...
3,1j6gaje,1.741437e+09,3,1,taste longer exists,hi everyone little quick context year old mtf ...
4,1j6f3dt,1.741433e+09,1,1,feel like im burning inside im tired,feel severely depressed since woke everything ...


In [8]:
df[df["Upvotes"]>100].count()

ID                 15
Timestamp          15
Upvotes            15
Comments           15
Cleaned_Title      15
Cleaned_Content    14
dtype: int64

In [10]:
df.shape

(1864, 6)

In [12]:
df = df.drop_duplicates(subset=["Cleaned_Title", "Cleaned_Content"], keep="first")
df.shape

(1844, 6)

In [13]:
df.isnull().sum()

ID                  0
Timestamp           0
Upvotes             0
Comments            0
Cleaned_Title       9
Cleaned_Content    11
dtype: int64

In [14]:
df = df.dropna(subset=["Cleaned_Title", "Cleaned_Content"])


In [15]:
df.shape

(1824, 6)

In [16]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit="s")


In [ ]:
df.to_csv("Cleaned_dataset")